In [109]:
import pandas as pd
import re
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import swifter

Import Dataset CSV with Pandas

In [110]:
df = pd.read_csv("Dataset\dataset_siap.csv")
# print(df)

Lower Case with Pandas

In [111]:
df = df.astype(str).apply(lambda x: x.str.lower())
print(df)

                                               Kalimat Kelas
0         tampilkan kartu hasil studi untuk semester 1   id 
1         tampilkan kartu hasil studi untuk semester 2   id 
2         tampilkan kartu hasil studi untuk semester 3   id 
3         tampilkan kartu hasil studi untuk semester 4   id 
4         tampilkan kartu hasil studi untuk semester 5   id 
..                                                 ...   ...
325  apa yang menjadi alasan/landasan anda dalam me...   ood
326           seberapa penting bisa berbahasa inggris?   ood
327     kapan tere liye menerbitkan buku pertama kali?   ood
328                         mengapa kita perlu minum ?   ood
329         siapa penulis puisi pada suatu hari nanti?   ood

[330 rows x 2 columns]


Remove Puncutuation

In [87]:
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text

df['Kalimat']= df['Kalimat'].apply(clean_punct)


In [112]:
print(df)

                                               Kalimat Kelas
0         tampilkan kartu hasil studi untuk semester 1   id 
1         tampilkan kartu hasil studi untuk semester 2   id 
2         tampilkan kartu hasil studi untuk semester 3   id 
3         tampilkan kartu hasil studi untuk semester 4   id 
4         tampilkan kartu hasil studi untuk semester 5   id 
..                                                 ...   ...
325  apa yang menjadi alasan/landasan anda dalam me...   ood
326           seberapa penting bisa berbahasa inggris?   ood
327     kapan tere liye menerbitkan buku pertama kali?   ood
328                         mengapa kita perlu minum ?   ood
329         siapa penulis puisi pada suatu hari nanti?   ood

[330 rows x 2 columns]


Tekonize with NLTK

In [113]:
df['Kalimat'] = df.apply(lambda row: nltk.word_tokenize(row['Kalimat']), axis=1)
print(df)

                                               Kalimat Kelas
0    [tampilkan, kartu, hasil, studi, untuk, semest...   id 
1    [tampilkan, kartu, hasil, studi, untuk, semest...   id 
2    [tampilkan, kartu, hasil, studi, untuk, semest...   id 
3    [tampilkan, kartu, hasil, studi, untuk, semest...   id 
4    [tampilkan, kartu, hasil, studi, untuk, semest...   id 
..                                                 ...   ...
325  [apa, yang, menjadi, alasan/landasan, anda, da...   ood
326   [seberapa, penting, bisa, berbahasa, inggris, ?]   ood
327  [kapan, tere, liye, menerbitkan, buku, pertama...   ood
328                   [mengapa, kita, perlu, minum, ?]   ood
329  [siapa, penulis, puisi, pada, suatu, hari, nan...   ood

[330 rows x 2 columns]


Stopwords with NLTK

In [114]:
from nltk.corpus import stopwords
list_stopwords = stopwords.words('indonesian')

def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['Kalimat'] = df['Kalimat'].apply(stopwords_removal)

Stemming with Sastrawi

In [115]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)
term_dict = {}

for document in df['Kalimat']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)


In [125]:

def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

df["TF_dict"] = df['Kalimat'].apply(calc_TF)

df["TF_dict"].head()

0    {'tampil': 0.16666666666666666, 'kartu': 0.166...
1    {'tampil': 0.16666666666666666, 'kartu': 0.166...
2    {'tampil': 0.16666666666666666, 'kartu': 0.166...
3    {'tampil': 0.16666666666666666, 'kartu': 0.166...
4    {'tampil': 0.16666666666666666, 'kartu': 0.166...
Name: TF_dict, dtype: object

In [130]:
df

,Kalimat,Kelas,Kalimat_join,TF_dict,TF-IDF_dict
0,"[tampil, kartu, hasil, studi, semester, 1]",id,tampil kartu hasil studi semester,"{'tampil': 0.16666666666666666, 'kartu': 0.166...","{'tampil': 0.15525970066749056, 'kartu': 0.552..."
1,"[tampil, kartu, hasil, studi, semester, 2]",id,tampil kartu hasil studi semester,"{'tampil': 0.16666666666666666, 'kartu': 0.166...","{'tampil': 0.15525970066749056, 'kartu': 0.552..."
2,"[tampil, kartu, hasil, studi, semester, 3]",id,tampil kartu hasil studi semester,"{'tampil': 0.16666666666666666, 'kartu': 0.166...","{'tampil': 0.15525970066749056, 'kartu': 0.552..."
3,"[tampil, kartu, hasil, studi, semester, 4]",id,tampil kartu hasil studi semester,"{'tampil': 0.16666666666666666, 'kartu': 0.166...","{'tampil': 0.15525970066749056, 'kartu': 0.552..."
4,"[tampil, kartu, hasil, studi, semester, 5]",id,tampil kartu hasil studi semester,"{'tampil': 0.16666666666666666, 'kartu': 0.166...","{'tampil': 0.15525970066749056, 'kartu': 0.552..."
...,...,...,...,...,...
325,"[alas landas, chatbot]",ood,alas landas chatbot,"{'alas landas': 0.5, 'chatbot': 0.5}","{'alas landas': 2.55297273695029, 'chatbot': 2..."
326,"[bahasa, inggris, ]",ood,bahasa inggris,"{'bahasa': 0.3333333333333333, 'inggris': 0.33...","{'bahasa': 1.4709327644468781, 'inggris': 1.56..."
327,"[tere, liye, terbit, buku, kali, ]",ood,tere liye terbit buku kali,"{'tere': 0.16666666666666666, 'liye': 0.166666...","{'tere': 0.6982757903377375, 'liye': 0.6982757..."
328,"[minum, ]",ood,minum,"{'minum': 0.5, '': 0.5}","{'minum': 2.3502401828962083, '': 0.3143043297..."


ValueError: malformed node or string: nan

In [136]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tfidf = TfidfVectorizer(max_features=500)
X_tfidf = vectorizer_tfidf.fit_transform(df['Kalimat'])
print(vectorizer_tfidf.get_feature_names())

AttributeError: 'list' object has no attribute 'lower'